# Question 4
Using image of numbers classify them using beyes theorm. Note that here the parameters of guassian are unknown so with parameter estimation methods such as MLE, first calculate the parameters of guassian distribution and then use the beyes theorm to classify images. Divide images dataset into half training and half test set.

Going through the MLE (Maximum likelihood) method we calculate the $p(D|\theta)$ and $D = {x_1, x_2, ..., x_n}$. To find the $\hat{\theta}_{ML}$, we need to find the max argument of $p(D|\theta)$. As the eqaution below:
$$
\begin{equation}
\hat{\theta}_{ML} = argmax \; p(D|\theta) =>  p({x_1, x_2, ..., x_n} | \theta) = \prod_{i=1}^{n} p(x_i | \theta)
\end{equation} 
$$
and if we continue the equation above we will find the argmax as the derivative of $p(D|\theta)$ with respect to $\theta$.
$$
\begin{equation}
\hat{\theta}_{ML} = \prod_{i=1}^{n} p(x_i | \theta) \; d\theta = 0
\end{equation} 
$$
To make it more simple we can apply the logarithm to the equation to make the product as a summation
$$
\begin{equation}
\hat{\theta}_{ML} = ln \;(\prod_{i=1}^{n} p(x_i | \theta) \; d\theta) = 0 \quad => \quad \hat{\theta}_{ML} = \sum_{i=1}^{n} ln \; p(x_i | \theta) \; d\theta = 0
\end{equation} 
$$

We will choose the normal distribution function and using the equation 3 we will find the mean and covariance as below:
$$
\begin{equation}
\hat{\mu} = \frac{1}{n} \sum_{k=1}^{n} x_k
\end{equation}
$$
$$
\begin{equation}
\hat{\Sigma} =  \frac{1}{n} \sum_{k=1}^{n} (x_k - \hat{\mu}) (x_k - \hat{\mu})^t 
\end{equation}
$$
So lets get to work and first calculate the $\hat{\mu}$ and $\hat{\Sigma}$.

In [1]:
import numpy as np
import pandas as pd

In [2]:
## read dataset
dataset = pd.read_csv('../numbers dataset/usps_images.csv')
print(dataset.shape)
dataset.head()

(5610, 257)


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_247,feature_248,feature_249,feature_250,feature_251,feature_252,feature_253,feature_254,feature_255,label
0,0,8,0,0,6,2,0,4,0,11,...,4,5,0,0,1,0,0,0,6,0
1,0,0,9,0,4,0,6,6,0,10,...,4,8,0,0,9,0,4,8,0,0
2,1,0,0,0,0,2,4,4,0,5,...,4,9,0,0,2,0,0,0,1,0
3,0,0,0,0,3,4,2,0,0,4,...,9,6,0,5,1,0,0,0,5,0
4,0,3,0,0,9,0,5,2,1,0,...,0,0,2,4,3,0,0,0,0,0


In [3]:
def divide_dataframe(dataset, validation_split, label_string):
    """
    Divide a pandas dataframe into two dataframes

    INPUTS:
    ---------
    dataset:  the pandas dataframe that is going to be splited
    validation_split:  the partition of how the validation set will be, example: 0.1
    label:  For a multiclass classification, we need the label to get sample from each label (Must be an string)

    OUTPUTS:
    ---------
    train_set:  pandas dataframe, the partition of training set
    valid_set:  pandas dataframe, the partition of validation set  
    """

    assert ((validation_split >0) & (validation_split < 1)), "[ERROR] validation_split must be between 0 and 1!"
    columns = dataset.columns
    train_images = pd.DataFrame(columns=columns)
    validation_images = pd.DataFrame(columns=columns)

    ## get all labels for training and test data
    for label in dataset[label_string].unique():
        ## get the dataset for each dataset label
        dataset_label = dataset[dataset[label_string] == label]
        length = int(len(dataset_label) * validation_split)

        train_images = train_images.append(dataset_label.iloc[: length], ignore_index=True)
        validation_images = validation_images.append(dataset_label.iloc[length: ], ignore_index=True)

    features = columns[:len(columns) - 1]

    ## normalize the values into float
    ## we need to convert the integer values to float for KNN function
    for col in features:
        train_images[col] = pd.to_numeric(train_images[col], downcast='float')
        validation_images[col] = pd.to_numeric(validation_images[col], downcast='float')

    return train_images, validation_images

df_train, df_test = divide_dataframe(dataset, 0.5, 'label')

In [128]:
def mu_hat(x_vectors):
    """"
    find the mu hat from Maximum likelihood, using the equation 4

    INPUTS:
    -------
    x_vectors:  a pandas dataframe of features

    OUTPUT:
    ---------
    mu_hat:  the mean for all data
    """

    sum = 0
    for i in range(0, len(x_vectors)):
        sum += x_vectors.iloc[i]
    
    mu_hat = (1 / len(x_vectors)) * sum

    return mu_hat

def Sigma_hat(mu, x_vectors):
    """
    calculate the covariance estimation using Maximum likelihood, the equation 5

    INPUTS:
    --------
    mu:  the mu value of each class
    x_vectors:  a pandas dataframe of features

    OUTPUT:
    --------
    Sigma:  the covarince matrixes of dataset
    """
    matrix = np.zeros((256, 256))
    matrix_values = []
    for i in range(0, len(x_vectors)):
        ## features must be a column vector
        X = x_vectors.iloc[i].T

        ## to make the array as column vector, we again need to transpose it
        subtract = np.array([X - mu]).T

        value = np.dot(subtract.T, subtract) 
        
        matrix_values += value

    return matrix


features = df_train.columns[df_train.columns != 'label']
train_mean = mu_hat(df_train[features])
test_mean = mu_hat(df_test[features])

train_Sigma = Sigma_hat(train_mean, df_train[features])
test_Sigma = Sigma_hat(test_mean, df_test[features])


print("TRAIN MEAN shape: ", train_mean.shape)
print("TEST MEAN shape: ", test_mean.shape )

print("train Sigma_hat shape: ", np.array(train_Sigma).shape)
print("test Sigma_hat shape: ", np.array(test_Sigma).shape)


TRAIN MEAN shape:  (256,)
TEST MEAN shape:  (256,)
train Sigma_hat shape:  (256, 256)
test Sigma_hat shape:  (256, 256)


It's time to create the classifier function. We will use the multivariate gaussian distribution function. We had written the function in Question 2 so we just coppied it. (we ade just some minor changes to support as many features as available )

In [131]:
## compute the probability of x using the x value
def guassian_multivariate(mu ,Sigma, X):
    """
    calculate the probability of multivariate guassian
    we have a feature space of 2 by 1

    INPUTS:
    --------
    mu:  is the mean value 
    Sigma:  covariance matrix
    X:  feature array

    OUTPUT:
    -------
    probability:  A scalar value representing the probability of X features 
    """
    # covariance determanant
    det_Sigma = np.linalg.det(Sigma)

    coefficient = 1 / 2 * np.pi

    ## inverse of covariance matrix
    if(det_Sigma != 0):
        Sigma_inv = np.linalg.inv(Sigma)

        ## exponential power value ( the superscript) 
        e_superscript =  (-1/2) * np.dot((X - mu).T , Sigma_inv ).dot(X - mu) 
        e_superscript = np.array(e_superscript, dtype=np.float128)

        e = np.exp(e_superscript)


        probability = coefficient * np.sqrt(det_Sigma) * e
    
    ## the probabilty of -1 means we couldn't find the inverse matrix
    else:
        probability = -1

    return probability



In [135]:
probabilities = []
for idx in range(0, len(df_train)):
    probability = guassian_multivariate(train_mean, train_Sigma, df_train[features].iloc[idx])
    probabilities.append(probability)
## check a portion of probabilities
probabilities[:5]

[-1, -1, -1, -1, -1]

**This part is NOT USEFULE!, was made before finding determinant of zero (will be removed in the next commit)** <br>
As we can see here we got overflow for it. So to find the probability for the classes we can apply the logarithm to it. 
The logarithm applied to guassian multivariate distribution function will be like below:
$$
\begin{equation}
g(x) = - \frac{1}{2} (x - \mu_i)^t \Sigma^{-1} (x-\mu_i) - \frac{d}{2} ln(2\pi) - \frac{1}{2} ln |\Sigma_i| 
\end{equation}
$$
<!-- Some of the formulation in the above equation is not necessery because we're comparing two classes and they are constants. So the equation we need to implement is:
$$
\begin{equation}
g(x) = - \frac{1}{2} (x - \mu_i)^t \Sigma^{-1} (x-\mu_i) - \frac{1}{2} ln |\Sigma_i| 
\end{equation}
$$ -->

In [120]:
## compute the probability of x using the x value
def guassian_multivariate_using_discriminant_function(mu ,Sigma, X):
    """
    calculate the probability of multivariate guassian using the logarithm discriminant function
    we have a feature space of 2 by 1

    INPUTS:
    --------
    mu:  is the mean value 
    Sigma:  covariance matrix
    X:  feature arrays

    OUTPUT:
    -------
    probability:  A scalar value representing the probability of X features 
    """
      
    
    Sigma_inv = np.linalg.inv(Sigma)

    part1 =  (-1/2) * np.dot((X - mu).T , Sigma_inv ).dot(X - mu) 

    # covariance determanant
    det_Sigma = np.linalg.det(Sigma)

    part2 = (-1/2) * np.log1p(2*np.pi) * len(X)

    part3 = (-1 /2) * np.log1p(det_Sigma)
        
    result = part1 + part2 + part3
    

    return result



In [121]:
# result_prob = []
# for idx in range(0, len(df_train)):
#     probability = guassian_multivariate(train_mean, np.array(train_Sigma, dtype=np.float64), df_train[features].iloc[idx])
#     result_prob.append(probability)
# np.array(result_prob).shape


/home/amin/.local/lib/python3.8/site-packages/numpy/linalg/linalg.py:2158: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [122]:
probability

array([[-inf, -inf, -inf, ..., -inf, -inf, -inf],
       [-inf, -inf, -inf, ..., -inf, -inf, -inf],
       [-inf, -inf, -inf, ..., -inf, -inf, -inf],
       ...,
       [-inf, -inf, -inf, ..., -inf, -inf, -inf],
       [-inf, -inf, -inf, ..., -inf, -inf, -inf],
       [-inf, -inf, -inf, ..., -inf, -inf, -inf]], dtype=float32)

Again we hitted with infinity values for results. This is the result of matrix determinant that is equal to infinity, So we need another method to calculate the parameters of our data.

In [ ]:
def check_classified_classes(results, dataset_label):
    """
    check positive or negative values for the result and return the accuracy

    INPUT:
    -------
    results:  numpy array contaning positive and negative values
    dataset_label:  numpy array containing the target classes

    OUTPUT:
    ---------
    accuracy:  the accuracy of the classification
    """

    

In [127]:
# A = np.log(np.array(train_Sigma, dtype=np.float32))
# A = A.round(2)
# np.linalg.det(A)
np.linalg.det(train_Sigma)

0.0